# **Tutorial for DICUnstructuredPoints model component**

DIC Grid serves for reading the data available as a sequence of files with displacement measurements stored in a directory, accompanied with the measured load deflection curve. It refers to the `BeamDesign` object defining the design of the reinforced beam. By combining the information from the optical measurement with statical scheme, geometry and reinforcement layout, this model component serves 
further analysis by providing the correct **load-space-time** databasis.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt

## Application example
Let us construct a grid example and render the interactive interface showing the measured displacmeent at the maximum load $t = 1$ and with displacement amplified by a factor $U_\mathrm{factor} = 100$. 

In [ ]:
from bmcs_shear.dic_crack.dic_inp_unstructured_points import DICInpUnstructuredPoints
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B1_TV1', t=1, padding=40, n_T_max=40, 
                                      T_stepping='delta_T')
dic_points.read_beam_design()
sz_bd = dic_points.sz_bd
sz_bd.system = 'simple_beam_dist_load'

In [ ]:
dic_points.sz_bd.interact()

In [ ]:
dic_points.interact()

Note that the orange markers denote the states with available DIC measurements performed during the history of loading.

## Read data format

### Input directory structure

The `DICGrid` objects reads the data from a directory named as `dir_name`

In [ ]:
dic_points.dir_name

The property trait `data_dir` assembles the path to the data directory using path starting in the home directory, i.e. 

```join(dic_grid.home_dir, 'simdb', 'data', 'shear_zone', dic_grid.dir_name)```

For the current example, we obtain

In [ ]:
dic_points.data_dir

To get the directory with the DIC data, issue 

In [ ]:
dic_points.dic_data_dir

### Load deflection response

To obtain the directory with the load deflection curve, use

In [ ]:
dic_points.time_F_w_data_dir

### Beam parameters
File specifying the beam parameters is assessed using the property traits `beam_param_fie` and is named `beam_params.txt` by default

In [ ]:
dic_points.beam_param_file

This file specifies the parameters 
 - length $L$
 - width $B$
 - depth $H$
 - number of reinforcement bars $n_\mathrm{s}$
 - vertical position of the bars $y_\mathrm{s}$
 - diameter of the reinforcement $d_\mathrm{s}$

In [ ]:
dic_points.beam_param_types

Currently, to setup the beam parmaters correctly, a call to `read_beam_design` must be issued using

In [ ]:
dic_points.read_beam_design()

**Note** This need should be avoided later by capturing the dependency with respect to the `data_dir` attribute. Currently, the notification mechanism does not allow this because of are further dependency links.

The beam design object is available in the attribute `sz_bd`. The type of the `RCBeamDesign` is general and specifies also the material models associated with individual material components.

The parameters relevant for correct configuration of the `dic_grid` within the beam design are the length and offset with respect the beam bounaries 

In [ ]:
dic_points.dic_data_dir

In [ ]:
dic_points.L_x, dic_points.L_y

### Grid parameters

File specifying the grid data parameters is accessed using the property trait `grid_param_file` and is named
`grid_params.txt` by default.

In [ ]:
dic_points.dic_param_file

This file contains the name-value pairs of the attributes defined by the `grid_param_types` dictionary

In [ ]:
dic_points.dic_param_types

Thus, the `grid_params` of the current test `B7_TV1` read

In [ ]:
dic_points.dic_params

The DIC data files corresponding to each measured step are assembled in the string list `files` 

These parameters are available as properties and can be accessed via

In [ ]:
dic_points.x_offset, dic_points.y_offset

Note that the naming of properties follows the mathematical indexing convention with $I$, $J$ representing the horizontal and vertical node indexes, respectively. Correspondingly, $n_I$ and $n_J$ denotes the number of horizontal and vertical nodes, respectively. 

To position the DIC frame into the beam, the beam parameters and grid parameters are combined to obtain the coordinates of the 
bottom left and top right corners within the beam in the order $X_\mathrm{frame} = (x_\min, y_\min, x_\max, y_\max)$

In [ ]:
dic_points.X_outer_frame

Note that the `CSV` files are named according to the scheme specifying their load levels

## Time data

Time `time` and load `F` corresponding to each history index `T` are provided in the `time_F_T` property.

In [ ]:
dic_points.time_F_T

The index corresponding to the maximum load `F` is accessed via

The final index of snapshots is also available as `n_T`

In [ ]:
dic_points.n_T

An instance of `dic_points` keeps track of the current time $t \in (0, 1)$ with 0 
and 1 representing the states with zero and maximum force, respectively. Thus, by setting

In [ ]:
dic_points.t = 0.5

The grid will return displacement and position values corresponding to the half of the maximum load. When setting the value of $t$, the value of `T1` denoting the history index will be adapted automateically, i.e.

In [ ]:
dic_points.T0, dic_points.n_T

The whole load and deflection values for each history index $T$ are available in the table `Fw_T`, representing the raw format of the measured data. The 

In [ ]:
time_m, F_m, w_m = dic_points.time_F_w_m

To access the deflections and forces, we need to issue

In [ ]:
w = w_m[::50]
F = -F_m[::50]

**TODO**: provide properties

# Implementation remark: DIC plane alignement

The next code snippet shows how to rotate the DIC plane orthogonal to the 
observation point so that $z$ values are nearly zero. This functionality 
is not yet implemented in the input component (as of 2022-01-12). If needed
the concept below can be used to included it.

In [ ]:
dic_points.X_outer_frame

In [ ]:
dic_points.X_inner_frame

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator
X_Qa = dic_points.X_Qa
points = X_Qa[:, :-1]
get_z = LinearNDInterpolator(Delaunay(X_Qa[:, :-1]), X_Qa[:, 2])

In [ ]:
bot_lft_Xa, top_rgt_Xa = dic_points.X_inner_frame
bot_rgt_Xa = np.array([top_rgt_Xa[0], bot_lft_Xa[1]])  
top_lft_Xa = np.array([bot_lft_Xa[0], top_rgt_Xa[1]])  
get_z(*bot_lft_Xa), get_z(*bot_rgt_Xa), get_z(*top_lft_Xa)

In [ ]:
X_plane_xy_Pa = np.array([ bot_lft_Xa, bot_rgt_Xa, top_lft_Xa ])
X_plane_z_Pa = np.array([ get_z(*X_a) for X_a in X_plane_xy_Pa ])
X_plane_Pa = np.hstack([X_plane_xy_Pa, X_plane_z_Pa[:, np.newaxis]])
X_plane_Pa

### Orthonormal basis

In [ ]:
EPS = np.zeros((3, 3, 3), dtype='f')
EPS[(0, 1, 2), (1, 2, 0), (2, 0, 1)] = 1
EPS[(2, 1, 0), (1, 0, 2), (0, 2, 1)] = -1
u_x_a = X_plane_Pa[1] - X_plane_Pa[0]
v_x_a = X_plane_Pa[2] - X_plane_Pa[0]
un_x_a = u_x_a / np.linalg.norm(u_x_a)
vn_x_a = v_x_a / np.linalg.norm(v_x_a)
w0_x_a = np.einsum('ijk,j,k', EPS, un_x_a, vn_x_a)
v0_x_a = np.einsum('ijk,j,k', EPS, w0_x_a, un_x_a)

In [ ]:
T_ab = np.array([un_x_a, v0_x_a, w0_x_a])
T_ab - T_ab.T

In [ ]:
X_orig_Ra = np.array([[0, 0, get_z(0, 0)]])
X_orig_Ra

In [ ]:
X_Ra = np.einsum('ba, Pa->Pb', T_ab, X_plane_Pa + X_orig_Ra)

In [ ]:
X_plane_Pa, X_Ra

In [ ]:
X_Ra = np.einsum('ba, Pa->Pb', T_ab, X_Qa)

In [ ]:
dic_points.t = 1
dic_points.F_T_t

In [ ]:
%matplotlib widget
from matplotlib.pyplot import figure,show
import numpy as np

fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.scatter3D(*X_Qa.T)
ax.scatter3D(*X_Ra.T)